In [1]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [2]:
import aiida.orm

geometry_node = aiida.orm.load_node(2501)

In [3]:
geometry_node.get_ase()

Atoms(symbols='C128H52', pbc=False, cell=[52.6424699363, 8.2124020701, 0.2005000008], masses=...)

In [4]:
from aiida.orm.code import Code
from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.base import Int, Float, Str, Bool
from aiida.work.run import submit
from aiida.orm.data.structure import StructureData

## -------------------------------------------------------
## Select computer and code
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})
qb.order_by({Code: {'id': 'asc'}})

computer, cp2k_code = qb.all()[1]

qb.all()
## -------------------------------------------------------

[[u'daint-s746',
  <Code: Remote code 'cp2k_6.0_18265' on daint-s746, pk: 214, uuid: f29f8268-2d86-4d8c-9fe8-cb491575db42>],
 [u'daint-s746-normal',
  <Code: Remote code 'cp2k_6.0_18265' on daint-s746-normal, pk: 1078, uuid: 2b831656-7e4e-4ede-b4a1-e7868ed86c34>]]

In [5]:
qb = QueryBuilder()
qb.append(Code)

stm_image_code = qb.all()[29][0]
eval_orbs_code = qb.all()[30][0]

for i, code in enumerate(qb.all()):
    print i, code

0 [<Code: Remote code 'cp2k_6.0_18265' on daint-s746, pk: 214, uuid: f29f8268-2d86-4d8c-9fe8-cb491575db42>]
1 [<Code: Remote code 'pw-6.1' on daint-s746, pk: 215, uuid: 72f53cb2-1725-444a-82ad-4d6b8b6b8d63>]
2 [<Code: Remote code 'pp-6.1' on daint-s746, pk: 216, uuid: 7828ae51-1a67-4675-97b6-09c35a92e00c>]
3 [<Code: Remote code 'ph-6.1' on daint-s746, pk: 217, uuid: aef87ad8-91c1-4c4f-a901-05851d3b4a69>]
4 [<Code: Remote code 'projwfc-6.1' on daint-s746, pk: 218, uuid: 049ab577-eea0-4f6a-bda4-9104c833839e>]
5 [<Code: Remote code 'matdyn-6.1' on daint-s746, pk: 219, uuid: 75226995-05f2-4f0b-accc-82fde07302fd>]
6 [<Code: Remote code 'q2r-6.1' on daint-s746, pk: 220, uuid: a59fde03-84a6-4b1a-9333-9ccc63a90c1c>]
7 [<Code: Remote code 'python3' on daint-s746-normal, pk: 2444, uuid: 8c7308b7-1cf8-4d53-b0d2-02ef70f345da>]
8 [<Code: Remote code 'cp2k_6.0_18265' on daint-s746-normal, pk: 1078, uuid: 2b831656-7e4e-4ede-b4a1-e7868ed86c34>]
9 [<Code: Remote code 'pw-6.1' on daint-s746-normal, pk: 

In [6]:
from aiida.orm.data.parameter import ParameterData

parent_dir = "parent_calc/"

eval_orbs_params = ParameterData(dict={
    '--cp2k_input':  parent_dir+'aiida.inp',
    '--basis_file':  parent_dir+'BASIS_MOLOPT',
    '--xyz_file':    parent_dir+'geom.xyz',
    '--wfn_file':    parent_dir+'aiida-RESTART.wfn',
    '--output_file': 'morbs_dx0.2',
    '--emin':        '-2.0',
    '--emax':        ' 2.0',
    '--eval_region': ['G', 'G', 'G', 'G', 'b-2.0_C', 't4.0'],
    '--dx':          '0.2',
    '--eval_cutoff': '14.0'
})

stm_image_params = ParameterData(dict={
    '--npz_file':          parent_dir+'morbs_dx0.2.npz',
    '--hartree_file':      parent_dir+parent_dir+'aiida-HART-v_hartree-1_0.cube',
    '--extrap_plane':      '3.0',
    '--extrap_extent':     '8.0',
    '--output_dir':        'stm_output',
#
    '--bias_voltages':     ['-1.0', '1.0'],
    '--stm_plane_heights': ['3.0', '5.0'],
    '--stm_isovalues':     ['1e-7', '1e-6'],
#
    '--sts_plane_heights': ['5.0'],
    '--sts_de':            '0.05',
    '--sts_fwhm':          '0.10',
    '--sts_elim':          ['-1.0', '1.0'],
#
    '--orb_plane_heights': ['3.0', '5.0'],
    '--n_homo':            '3',
    '--n_lumo':            '3'
})

In [7]:
from apps.stm.stm_workchain import STMWorkChain

In [8]:
outputs = submit(STMWorkChain,
                 cp2k_code=cp2k_code,
                 structure=geometry_node,
                 eval_orbs_code=eval_orbs_code,
                 eval_orbs_params=eval_orbs_params,
                 stm_image_code=stm_image_code,
                 stm_image_params=stm_image_params
                )
outputs

RunningInfo(type=<RunningType.PROCESS: 0>, pid=3087)

In [71]:
# Create inputs for the eval_morbs step

# Get the code from repo

! mkdir tmp
! wget https://github.com/eimrek/atomistic_tools/archive/master.zip -P ./tmp
! unzip ./tmp/master.zip -d ./tmp

mkdir: cannot create directory ‘tmp’: File exists
--2018-06-08 11:56:40--  https://github.com/eimrek/atomistic_tools/archive/master.zip
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/eimrek/atomistic_tools/zip/master [following]
--2018-06-08 11:56:41--  https://codeload.github.com/eimrek/atomistic_tools/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.253.121, 192.30.253.120
Connecting to codeload.github.com (codeload.github.com)|192.30.253.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘./tmp/master.zip.1’

master.zip.1            [    <=>             ]   7.48M  8.60MB/s    in 0.9s    

2018-06-08 11:56:42 (8.60 MB/s) - ‘./tmp/master.zip.1’ saved [7840482]

Archive:  ./tmp/master.zip
6a7e346c429dae792cbf34005159e

In [8]:
from aiida.orm.data.singlefile import SinglefileData
from aiida.orm.data.folder import FolderData


eval_script = SinglefileData(file="/project/apps/stm/tmp/atomistic_tools-master/evaluate_morbs_in_box_mpi.py")
eval_lib_folder = FolderData(file="/project/apps/stm/tmp/atomistic_tools-master/atomistic_tools")

ValueError: Unable to set 'file', no set_file method found